In [ ]:
import pandas as pd
import glob
from functools import reduce
import yaml

In [ ]:
CONSUMPTION_SIZE = 1000.0  # kWh
FLEXIBILITY = 1.0  # from 0.0 to 1.0

In [ ]:
with open("data/portfolios.yml", "r") as fp:
    portfolios = yaml.load(fp, Loader=yaml.SafeLoader)
portfolios

In [ ]:
def read_portfolio(portfolio) -> pd.DataFrame:
    price = (
        pd.read_csv(f"data/price/{portfolio['area']}.csv", usecols=["time", "price"])
        .assign(time=lambda df: pd.to_datetime(df.time, utc=True))
        .set_index("time")
    )

    co2 = (
        pd.read_csv(f"data/co2/{portfolio['area']}.csv", usecols=["time", "co2"])
        .assign(time=lambda df: pd.to_datetime(df.time, utc=True))
        .filter(["time", "co2"], axis=1)
        .set_index("time")
    )

    data = []
    for site in portfolio["sites"]:
        path = f"data/sites/{site}.csv"
        df = (
            pd.read_csv(
                path,
                comment="#",
                usecols=["time", "electricity"]
            )
            .assign(time=lambda df: pd.to_datetime(df.time, format="%Y-%m-%d %H:%M", utc=True).dt.tz_convert("Europe/Berlin"))
            .set_index("time").resample("H")
            .mean().ffill()
        )
        
        data.append(df)
    
    output = (
        reduce(lambda a, b: a + b, data)
        .rename({"electricity": "res"}, axis=1)
        .merge(price, how="inner", left_index=True, right_index=True)
        .merge(co2, how="inner", left_index=True, right_index=True)
    )
    return output


In [ ]:
def compute_summary(results: pd.DataFrame) -> pd.DataFrame:
    co2_flat_total = results.co2_flat.sum()
    co2_profile_total = results.co2_profile.sum()
    co2_savings = co2_flat_total - co2_profile_total
    co2_savings_pct = co2_savings / co2_flat_total * 100.0

    cost_flat_total = results.cost_flat.sum()
    cost_profile_total = results.cost_profile.sum()
    cost_savings = cost_flat_total - cost_profile_total
    cost_savings_pct = cost_savings / cost_flat_total * 100.0

    summary = pd.DataFrame([
        dict(label="Flat CO2 (g CO2/MWh)", value=co2_flat_total),
        dict(label="Tracking CO2 (g CO2/MWh)", value=co2_profile_total),
        dict(label="CO2 reduction (g CO2/MWh)", value=co2_savings),
        dict(label="CO2 reduction (%)", value=co2_savings_pct),

        dict(label="Flat cost (EUR/MWh)", value=cost_flat_total),
        dict(label="Cost tracking (EUR/MWh)", value=cost_profile_total),
        dict(label="Cost reduction (EUR/KWh)", value=cost_savings),
        dict(label="Cost reduction (%)", value=cost_savings_pct)
    ])
    return summary

In [ ]:
for portfolio in portfolios:
    data = (
        read_portfolio(portfolio)
        .dropna()
        .assign(
            res=lambda df: df.res / df.res.sum() * CONSUMPTION_SIZE,
            flat=lambda df: CONSUMPTION_SIZE / len(df),
            profile=lambda df: (1.0 - FLEXIBILITY) * df.flat + FLEXIBILITY * df.res
        )
    )

    results = data.assign(
        co2_profile=lambda df: df.profile * df.co2,
        co2_flat=lambda df: df.flat * df.co2,
        cost_profile=lambda df: df.profile * df.price,
        cost_flat=lambda df: df.flat * df.price
    )
    results = results.reset_index().assign(time=lambda df: df.time.dt.tz_localize(None))
    summary = compute_summary(results)

    with pd.ExcelWriter(f"output/reports/{portfolio['name']}.xlsx") as writer:
        summary.to_excel(writer, sheet_name="Summary", index=False, header=False)
        results.to_excel(writer, sheet_name="Data", index=False)